# 7. ClassWeigh

In [1]:
import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datasets import Dataset

import mlflow

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    ConfusionMatrixDisplay,
)

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback,
    pipeline
)

import torch
from torch.nn import CrossEntropyLoss

print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available.")


c:\Users\kimdo\anaconda3\envs\yt_nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU count: 1
GPU name: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
df = pd.read_csv("../data/cleaned.csv")
df = df[["comment", "label_encoded"]].rename(columns={"comment": "text", "label_encoded": "label"})

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

In [ ]:


class ExperimentRunner:

    def __init__(self, train_ds, test_ds, num_labels=6, experiment_name="YouTubeCommentClassifier"):
        """
        Initialize Experiment Runner with datasets and MLflow setup.
        """
        self.train_ds = train_ds
        self.test_ds = test_ds
        self.num_labels = num_labels
        self.experiment_name = experiment_name

        mlflow.set_tracking_uri("file:../mlruns")
        mlflow.set_experiment(experiment_name)

    def compute_metrics(self, pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        acc = accuracy_score(labels, preds)
        return {"accuracy": acc}

    def _tokenize(self, batch):
        return self.tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)

    def train_and_evaluate(self, model_name, learning_rate=2e-5, batch_size=8, num_epochs=5, patience=2):
        """
        Train, evaluate, and log an experiment to MLflow.
        """
        if mlflow.active_run():
            mlflow.end_run()

        print(f"\n🚀 Running experiment: {model_name} | LR={learning_rate} | BS={batch_size}")

        experiment_dir = f"../experiment_results/{model_name.replace('/', '_')}_lr{learning_rate}_bs{batch_size}_ep{num_epochs}"
        os.makedirs(experiment_dir, exist_ok=True)


        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=self.num_labels)


        train_enc = self.train_ds.map(self._tokenize, batched=True)
        test_enc = self.test_ds.map(self._tokenize, batched=True)
        train_enc.set_format("torch", columns=["input_ids", "attention_mask", "label"])
        test_enc.set_format("torch", columns=["input_ids", "attention_mask", "label"])

        labels = [int(x) for x in self.train_ds["label"]]
        class_counts = torch.bincount(torch.tensor(labels))
        total_samples = float(sum(class_counts))
        class_weights = total_samples / (len(class_counts) * class_counts)
        class_weights = class_weights.to(torch.float32)
        print("Class weights:", class_weights.tolist())


        training_args = TrainingArguments(
            output_dir=experiment_dir,
            eval_strategy="epoch",
            save_strategy="epoch",
            learning_rate=learning_rate,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            num_train_epochs=num_epochs,
            weight_decay=0.01,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            greater_is_better=True,
            logging_dir="./logs",
            logging_steps=50,
        )

        # ---------------------------------------------------------
        #  Custom Trainer with Weighted Loss
        # ---------------------------------------------------------
        class WeightedTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
                labels = inputs.get("labels")
                outputs = model(**inputs)
                logits = outputs.get("logits")
                loss_fct = CrossEntropyLoss(weight=class_weights.to(model.device))
                loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                return (loss, outputs) if return_outputs else loss

        # ---------------------------------------------------------
        # Training and evaluation with MLflow logging
        # ---------------------------------------------------------
        with mlflow.start_run(run_name=f"{model_name}_lr{learning_rate}_bs{batch_size}"):

            mlflow.log_param("model_name", model_name)
            mlflow.log_param("learning_rate", learning_rate)
            mlflow.log_param("batch_size", batch_size)
            mlflow.log_param("epochs", num_epochs)
            mlflow.log_param("patience", patience)
            mlflow.log_param("class_weights", class_weights.tolist())

            trainer = WeightedTrainer(
                model=model,
                args=training_args,
                train_dataset=train_enc,
                eval_dataset=test_enc,
                tokenizer=self.tokenizer,
                compute_metrics=self.compute_metrics,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=patience)],
            )

            trainer.train()
            results = trainer.evaluate()

            for k, v in results.items():
                mlflow.log_metric(k, v)

            # -----------------------------------------------------
            # Evaluation & Visualization
            # -----------------------------------------------------
            print("Generating classification report and confusion matrix ...")
            preds_output = trainer.predict(test_enc)
            y_true = preds_output.label_ids
            y_pred = preds_output.predictions.argmax(-1)

            report = classification_report(y_true, y_pred, digits=3)
            report_path = os.path.join(
                experiment_dir,
                f"classification_report_{model_name.replace('/', '_')}_lr{learning_rate}_bs{batch_size}_ep{num_epochs}.txt",
            )
            with open(report_path, "w") as f:
                f.write(report)
            mlflow.log_artifact(report_path)

            cm = confusion_matrix(y_true, y_pred)
            plt.figure(figsize=(8, 6))
            sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
            plt.xlabel("Predicted")
            plt.ylabel("True")
            plt.title(f"{model_name} Confusion Matrix")
            cm_path = os.path.join(
                experiment_dir,
                f"confusion_matrix_{model_name.replace('/', '_')}_lr{learning_rate}_bs{batch_size}_ep{num_epochs}.png",
            )
            plt.tight_layout()
            plt.savefig(cm_path)
            mlflow.log_artifact(cm_path)
            plt.close()

            history = pd.DataFrame(trainer.state.log_history)
            train_loss = history[history["loss"].notna()][["epoch", "loss"]]
            eval_loss = history[history["eval_loss"].notna()][["epoch", "eval_loss"]]

            if not train_loss.empty and not eval_loss.empty:
                plt.figure(figsize=(8, 5))
                plt.plot(train_loss["epoch"], train_loss["loss"], marker="o", label="Training Loss")
                plt.plot(eval_loss["epoch"], eval_loss["eval_loss"], marker="s", label="Validation Loss")
                plt.legend()
                plt.title(f"{model_name} - LR={learning_rate}")
                plt.xlabel("Epoch")
                plt.ylabel("Loss")
                plt.tight_layout()
                plot_path = os.path.join(
                    experiment_dir,
                    f"loss_curve_{model_name.replace('/', '_')}_lr{learning_rate}_bs{batch_size}_ep{num_epochs}.png",
                )
                plt.savefig(plot_path)
                mlflow.log_artifact(plot_path)
                plt.close()

        print(f"✅ Experiment complete. Results saved in: {experiment_dir}\n")


In [4]:
models = ["distilbert-base-uncased", "bert-base-uncased", "roberta-base"]

learning_rates = [2e-5, 3e-5]
batch_sizes = [8, 16]
epochs = [5]
runner = ExperimentRunner(train_ds, test_ds)
for model_name, lr, bs, ep in itertools.product(models, learning_rates, batch_sizes, epochs):
    runner.train_and_evaluate(model_name, learning_rate=lr, batch_size=bs, num_epochs=ep)



🚀 Running experiment: distilbert-base-uncased | LR=2e-05 | BS=8


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 292/292 [00:00<00:00, 16197.22 examples/s]


Class weights: [2.0656027793884277, 1.981292486190796, 1.5051679611206055, 0.26203328371047974, 3.883333444595337, 3.6635220050811768]


C:\Users\kimdo\AppData\Local\Temp\ipykernel_19640\1532451395.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,1.602700,1.128897,0.753425
2,0.900500,0.817178,0.801370
3,0.499800,0.767462,0.832192
4,0.431700,0.774277,0.835616
5,0.325900,0.793900,0.856164


...

Generating classification report and confusion matrix ...
✅ Experiment complete. Results saved in: ../experiment_results/roberta-base_lr3e-05_bs8_ep5


🚀 Running experiment: roberta-base | LR=3e-05 | BS=16


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 292/292 [00:00<00:00, 9983.51 examples/s]
C:\Users\kimdo\AppData\Local\Temp\ipykernel_19640\1532451395.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(


Class weights: [2.0656027793884277, 1.981292486190796, 1.5051679611206055, 0.26203328371047974, 3.883333444595337, 3.6635220050811768]


Epoch,Training Loss,Validation Loss,Accuracy
1,1.779700,1.019580,0.664384
2,1.145700,0.735620,0.787671
3,0.424100,0.757684,0.760274
4,0.359800,0.788700,0.839041
5,0.213200,0.814869,0.839041


Generating classification report and confusion matrix ...
✅ Experiment complete. Results saved in: ../experiment_results/roberta-base_lr3e-05_bs16_ep5



In [ ]:
model_path = "../experiment_results/bert-base-uncased_lr2e-05_bs8_ep5/checkpoint-730"

output_dir = "../final_model"
os.makedirs(output_dir, exist_ok=True)

print(f"Loading model from: {model_path}")
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

pickle_path = os.path.join(output_dir, "final_model.pkl")
with open(pickle_path, "wb") as f:
    pickle.dump({"model": model, "tokenizer": tokenizer}, f)

print(f"Model and tokenizer successfully saved → {pickle_path}")


🔍 Loading model from: ../experiment_results/bert-base-uncased_lr2e-05_bs8_ep5/checkpoint-730
✅ Model and tokenizer successfully saved → ../experiment_results/final_model\final_model.pkl
